In [1]:
# ===== Eclipse – Cell 1: imports, mount, paths =====

import json
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import drive
drive.mount('/content/drive')

# Root where your v02 folder lives
DATA_ROOT = "/content/drive/MyDrive/msr2013-bug_dataset-master/data/v02"
ECLIPSE_DIR = f"{DATA_ROOT}/eclipse"

ASSIGNED_FILE   = f"{ECLIPSE_DIR}/assigned_to.json"
SHORT_DESC_FILE = f"{ECLIPSE_DIR}/short_desc.json"
BUG_STATUS_FILE = f"{ECLIPSE_DIR}/bug_status.json"
RESOLUTION_FILE = f"{ECLIPSE_DIR}/resolution.json"

print("Eclipse folder:", ECLIPSE_DIR)


Mounted at /content/drive
Eclipse folder: /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/eclipse


In [2]:
# ===== Eclipse – Cell 2 (FIXED): build classifier-like DataFrame from JSONs =====
import json
import pandas as pd

ASSIGNED_FILE   = f"{ECLIPSE_DIR}/assigned_to.json"
SHORT_DESC_FILE = f"{ECLIPSE_DIR}/short_desc.json"
BUG_STATUS_FILE = f"{ECLIPSE_DIR}/bug_status.json"
RESOLUTION_FILE = f"{ECLIPSE_DIR}/resolution.json"

print("Files:")
print("  ", ASSIGNED_FILE)
print("  ", SHORT_DESC_FILE)
print("  ", BUG_STATUS_FILE)
print("  ", RESOLUTION_FILE)

def load_last_event_table(path, root_key, out_col):
    """
    MSR2013 JSON format:
      { root_key : { bug_id : [ {when, what, who}, ... ], ... } }

    We take, for each bug_id, the LAST event (max 'when') and keep its 'what'.
    Returns a DataFrame with columns: ['bug_id', out_col].
    """
    with open(path, "r", encoding="utf-8") as f:
        raw = json.load(f)

    inner = raw[root_key]          # dict: bug_id -> list of events
    rows = []

    for bug_id, events in inner.items():
        if not events:
            continue
        # last event by time
        last = max(events, key=lambda e: e.get("when", 0))
        rows.append({
            "bug_id": int(bug_id),             # bug id as integer
            out_col: str(last.get("what", "")) # final value (email, summary, status, etc.)
        })

    return pd.DataFrame(rows)

# 2.1 flatten each JSON into (bug_id, value) tables
assigned_df   = load_last_event_table(ASSIGNED_FILE,   "assigned_to", "assigned_to")
short_df      = load_last_event_table(SHORT_DESC_FILE, "short_desc",  "short_desc")
bug_status_df = load_last_event_table(BUG_STATUS_FILE, "bug_status",  "bug_status")
resolution_df = load_last_event_table(RESOLUTION_FILE, "resolution",  "resolution")

print("assigned_df head:")
display(assigned_df.head())
print("short_df head:")
display(short_df.head())
print("bug_status_df head:")
display(bug_status_df.head())
print("resolution_df head:")
display(resolution_df.head())

# 2.2 merge on bug_id to get one row per bug
df = (assigned_df
      .merge(short_df,      on="bug_id", how="inner")
      .merge(bug_status_df, on="bug_id", how="inner")
      .merge(resolution_df, on="bug_id", how="inner"))

print("Merged rows before filters:", len(df))

# 2.3 filter to RESOLVED / VERIFIED / CLOSED **and** FIXED
status_upper = df["bug_status"].str.upper()
res_upper    = df["resolution"].str.upper()

mask_status = status_upper.isin(["RESOLVED", "VERIFIED", "CLOSED"])
mask_res    = res_upper.str.contains("FIXED", na=False)

df = df[mask_status & mask_res].copy()
print("Rows after resolved+fixed filter:", len(df))

# 2.4 build classifier-style columns: owner, issue_title, description
df["owner"]       = df["assigned_to"].astype(str)
df["issue_title"] = df["short_desc"].astype(str)
df["description"] = ""   # no long description in this folder

# keep only developers with >= 20 bugs (same spirit as classifier_data_20)
MIN_BUGS = 2
owner_counts = df["owner"].value_counts()
keep_owners  = owner_counts[owner_counts >= MIN_BUGS].index
df = df[df["owner"].isin(keep_owners)].reset_index(drop=True)

print("Rows after MIN_BUGS filter:", len(df))
print("Number of developers with ≥2 bugs:", df["owner"].nunique())

# 2.5 encode dev_id and create the classifier-style df used later
df["dev_id"] = df["owner"].astype("category").cat.codes
n_devs = df["dev_id"].nunique()
print("Final number of developers:", n_devs)

clf_df = df[["owner", "issue_title", "description", "dev_id"]]
display(clf_df.head())


Files:
   /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/eclipse/assigned_to.json
   /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/eclipse/short_desc.json
   /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/eclipse/bug_status.json
   /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/eclipse/resolution.json
assigned_df head:


bug_id           assigned_to
0    1609  Darin_Wright@oti.com
1    1536  Darin_Wright@oti.com
2    1537  Darin_Wright@oti.com
3    1538  Darin_Wright@oti.com
4    1539  Darin_Wright@oti.com

short_df head:


bug_id                                         short_desc
0    3641         DCR: Can't compare from outliner (1GDHJKK)
1    3470  DCR: Can't compare code editions in type hiera...
2    3337  Open on selection failed with double message (...
3    3210  Search - method declarations within TypeHierar...
4    2968              TaskListMarker violates API (1GL2XHA)

bug_status_df head:


bug_id bug_status
0    4203   VERIFIED
1    4351   RESOLVED
2    3456     CLOSED
3    3475   VERIFIED
4    3469   RESOLVED

resolution_df head:


bug_id resolution
0    4203      FIXED
1    4351      FIXED
2    3456      FIXED
3    3475      FIXED
4    3469    WONTFIX

Merged rows before filters: 165547
Rows after resolved+fixed filter: 76268
Rows after MIN_BUGS filter: 76185
Number of developers with ≥2 bugs: 439
Final number of developers: 439


owner                                        issue_title  \
0     Darin_Wright@oti.com  classFilePattern breakpoint attribute optimiza...   
1      Jared_Burns@oti.com                       NPE in LocalEvaluationEngine   
2  Joe_Szurszewski@oti.com                   Extra Action delegates (1GIGUK0)   
3    Darin_Swanson@oti.com              HotSpot VM -server argument (1GLCPDH)   
4    Darin_Swanson@oti.com    Code Review: JDI debug model shutdown (1GLCWH3)   

  description  dev_id  
0                  23  
1                  40  
2                  46  
3                  20  
4                  20

In [3]:
# ==== Cell 3 – tokenisation + train/val/test splits (Eclipse) ====

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

MAX_VOCAB  = 50000
MAX_LEN    = 300       # same as Chromium
BATCH_SIZE = 256
VAL_SIZE   = 0.1       # 10% of train_full used as validation

# 3.1 build a single text field (title + description)
clf_df["text"] = (
    clf_df["issue_title"].fillna("").astype(str)
    + " "
    + clf_df["description"].fillna("").astype(str)
)

# 3.2 labels (developer ids)
y_all = clf_df["dev_id"].astype("int32").values
n_devs = clf_df["dev_id"].nunique()
print("Number of developers:", n_devs)

texts = clf_df["text"].tolist()

# 3.3 Keras tokenizer on all Eclipse text
tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
seqs = tokenizer.texts_to_sequences(texts)

X_all = pad_sequences(
    seqs,
    maxlen=MAX_LEN,
    padding="post",
    truncating="post"
)
print("X_all shape:", X_all.shape)

# 3.4 stratified 80/20 train+test split
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_all,
    y_all,
    test_size=0.2,
    random_state=42,
    stratify=y_all,
)

# 3.5 from train_full, carve out validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full,
    y_train_full,
    test_size=VAL_SIZE,
    random_state=42,
    stratify=y_train_full,
)

print("Train:", X_train.shape, " Val:", X_val.shape, " Test:", X_test.shape)

# 3.6 vocab size actually used
vocab_size = min(MAX_VOCAB, len(tokenizer.word_index) + 1)
print("Vocab size (capped):", vocab_size)


Number of developers: 439


/tmp/ipython-input-1983104502.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clf_df["text"] = (


X_all shape: (76185, 300)
Train: (54853, 300)  Val: (6095, 300)  Test: (15237, 300)
Vocab size (capped): 26957


In [4]:
# ==== Cell 4 – DBRNN model (BiLSTM + attention) for Eclipse ====

import tensorflow as tf
from tensorflow.keras import layers, models

EMBED_DIM  = 128
LSTM_UNITS = 128

def build_dbrnn_model(vocab_size, max_len, n_classes):
    inputs = layers.Input(shape=(max_len,), dtype="int32")

    # We keep mask_zero=False to avoid complications with attention
    x = layers.Embedding(
        input_dim=vocab_size,
        output_dim=EMBED_DIM,
        mask_zero=False
    )(inputs)

    # Bi-LSTM over the sequence
    x = layers.Bidirectional(
        layers.LSTM(LSTM_UNITS, return_sequences=True)
    )(x)   # (B, T, 2*LSTM_UNITS)

    # --- Simple additive attention ---
    e = layers.Dense(1, activation="tanh")(x)           # (B, T, 1)

    alpha = layers.Lambda(
        lambda t: tf.nn.softmax(t, axis=1),
        name="attn_weights"
    )(e)                                                # (B, T, 1)

    def apply_attention(tensors):
        h, a = tensors   # h: (B, T, 2H), a: (B, T, 1)
        return tf.reduce_sum(h * a, axis=1)   # (B, 2H)

    doc_repr = layers.Lambda(apply_attention, name="doc_repr")([x, alpha])

    outputs = layers.Dense(n_classes, activation="softmax")(doc_repr)

    model = models.Model(inputs=inputs, outputs=outputs, name="DBRNN_Eclipse")
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

n_classes = n_devs
model = build_dbrnn_model(vocab_size, MAX_LEN, n_classes)
model.summary()


Model: "DBRNN_Eclipse"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 300, 128)  │  3,450,496 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 300, 256)  │    263,168 │ embedding[0][0]   │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 300, 1)    │        257 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attn_weights        │ (None, 300, 1)    │          0 │ dense[0][0]       │
│ (Lambda)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ doc_repr (Lambda)   │ (None, 256)       │          0 │ bidirectional[0]… │
│                     │                   │            │ attn_weights[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 439)       │    112,823 │ doc_repr[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,826,744 (14.60 MB)

 Trainable params: 3,826,744 (14.60 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# ==== Cell 5 – train DBRNN on Eclipse ====

EPOCHS = 10   # with early stopping; tends to give HR ~ Chromium

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=2,
        restore_best_weights=True
    )
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    callbacks=callbacks,
)


Epoch 1/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 26s 84ms/step - accuracy: 0.2492 - loss: 4.5875 - val_accuracy: 0.2561 - val_loss: 4.3563
Epoch 2/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 19s 91ms/step - accuracy: 0.2542 - loss: 4.3604 - val_accuracy: 0.2561 - val_loss: 4.2446
Epoch 3/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 18s 85ms/step - accuracy: 0.2570 - loss: 4.1706 - val_accuracy: 0.2632 - val_loss: 3.9381
Epoch 4/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 18s 85ms/step - accuracy: 0.2733 - loss: 3.7675 - val_accuracy: 0.2722 - val_loss: 3.5993
Epoch 5/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 18s 85ms/step - accuracy: 0.2977 - loss: 3.3681 - val_accuracy: 0.2833 - val_loss: 3.4135
Epoch 6/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 18s 86ms/step - accuracy: 0.3208 - loss: 3.0689 - val_accuracy: 0.2820 - val_loss: 3.3484
Epoch 7/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 18s 85ms/step - accuracy: 0.3502 - loss: 2.8158 - val_accuracy: 0.2940 - val_loss: 3.3018
Epoch 8/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 18s 85ms/step - accuracy: 0.3776 - loss: 2.6196 - 

In [6]:
# ==== Cell 6 – DBRNN baseline evaluation (Hit@K) on Eclipse ====

import numpy as np

def hit_at_k(probs, y_true, k):
    topk = np.argsort(-probs, axis=1)[:, :k]
    hits = (topk == y_true.reshape(-1, 1)).any(axis=1)
    return float(hits.mean())

# Predict probabilities on Eclipse test set
probs_base = model.predict(X_test, batch_size=BATCH_SIZE)

TOPKS = [1, 3, 5, 7, 9]
hrs_dbrnn = []

print("=== DBRNN baseline (Eclipse, no graph) ===")
for k in TOPKS:
    hr = hit_at_k(probs_base, y_test, k)
    hrs_dbrnn.append(hr)
    print(f"Top-{k} Hit Ratio: {hr:.4f}")

avg_hr_dbrnn = float(np.mean(hrs_dbrnn))
print(f"Average Hit Ratio DBRNN : {avg_hr_dbrnn:.4f}")


60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step
=== DBRNN baseline (Eclipse, no graph) ===
Top-1 Hit Ratio: 0.2840
Top-3 Hit Ratio: 0.4840
Top-5 Hit Ratio: 0.5772
Top-7 Hit Ratio: 0.6400
Top-9 Hit Ratio: 0.6837
Average Hit Ratio DBRNN : 0.5338


In [7]:
# ==== Cell 7 – developer vectors + adjacency matrix (Eclipse) ====

from sklearn.metrics.pairwise import cosine_similarity

# 7.1 – bug representations from the "doc_repr" layer
repr_model = models.Model(
    inputs=model.input,
    outputs=model.get_layer("doc_repr").output
)

bug_repr_all = repr_model.predict(X_all, batch_size=BATCH_SIZE)
repr_dim = bug_repr_all.shape[1]
print("Bug repr shape:", bug_repr_all.shape)

# 7.2 – aggregate to developer-level vectors (mean of their bugs)
dev_repr   = np.zeros((n_devs, repr_dim), dtype=np.float32)
dev_counts = np.zeros(n_devs, dtype=np.int32)

for i, d in enumerate(y_all):
    dev_repr[d] += bug_repr_all[i]
    dev_counts[d] += 1

nonzero = dev_counts > 0
dev_repr[nonzero] /= dev_counts[nonzero][:, None]

print("Developer repr shape:", dev_repr.shape)

# 7.3 – cosine similarity graph + k-NN sparsification
K_NEIGHBORS = 20  # same tuning as Chromium (dense but not fully connected)

sim = cosine_similarity(dev_repr)
np.fill_diagonal(sim, 0.0)

A = np.zeros_like(sim, dtype=np.float32)
for i in range(n_devs):
    idx = np.argsort(-sim[i])[:K_NEIGHBORS]
    A[i, idx] = sim[i, idx]

# Symmetrise
A = np.maximum(A, A.T)

# 7.4 – row-normalise and add self-loops
row_sums = A.sum(axis=1, keepdims=True) + 1e-8
A_norm = A / row_sums

SELF_LOOP = 0.6  # 60% keep own prob, 40% neighbours
A_smooth = SELF_LOOP * np.eye(n_devs, dtype=np.float32) + (1.0 - SELF_LOOP) * A_norm

print("A_smooth shape:", A_smooth.shape)


298/298 ━━━━━━━━━━━━━━━━━━━━ 9s 31ms/step
Bug repr shape: (76185, 256)
Developer repr shape: (439, 256)
A_smooth shape: (439, 439)


In [8]:
# ==== Cell 8 – GRCNN-style smoothing (Eclipse) ====

STEPS = 2   # propagation steps, same as Chromium

print("=== GRCNN-style (DBRNN + developer graph smoothing) on Eclipse ===")

probs_graph = probs_base.copy()
for _ in range(STEPS):
    probs_graph = np.matmul(probs_graph, A_smooth)

TOPKS = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
hrs_grcnn = []

for k in TOPKS:
    hr_g = hit_at_k(probs_graph, y_test, k)
    hrs_grcnn.append(hr_g)
    print(f"Top-{k} Hit Ratio (graph): {hr_g:.4f}")

avg_hr_graph = float(np.mean(hrs_grcnn))
print(f"Average Hit Ratio GRCNN : {avg_hr_graph:.4f}")

print("\nSummary on Eclipse:")
print(f"  DBRNN  avg HR : {avg_hr_dbrnn:.4f}")
print(f"  GRCNN* avg HR : {avg_hr_graph:.4f}  (GRCNN-style smoothing)")


=== GRCNN-style (DBRNN + developer graph smoothing) on Eclipse ===
Top-1 Hit Ratio (graph): 0.2801
Top-2 Hit Ratio (graph): 0.4012
Top-3 Hit Ratio (graph): 0.4679
Top-4 Hit Ratio (graph): 0.5142
Top-5 Hit Ratio (graph): 0.5499
Top-6 Hit Ratio (graph): 0.5770
Top-7 Hit Ratio (graph): 0.5998
Top-8 Hit Ratio (graph): 0.6184
Top-9 Hit Ratio (graph): 0.6346
Top-10 Hit Ratio (graph): 0.6480
Average Hit Ratio GRCNN : 0.5291

Summary on Eclipse:
  DBRNN  avg HR : 0.5338
  GRCNN* avg HR : 0.5291  (GRCNN-style smoothing)
